In [23]:
# Voer dit blok code uit door met de cursor in het blok te klikken, en vervolgens SHIFT+ENTER in te drukken.

import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import odeint

from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, FloatLogSlider, Layout


%matplotlib inline

style = {'description_width': '200px'}
slider_layout = Layout(width='50%')

# T3 - Afgeleiden
## T3.1 - Introductie

Het begrip *afgeleide* komt frequent voor in allerlei wetenschappelijke en andere contexten. De techniek van afgeleiden laat toe om precies, efficiënt en in zeer uiteenlopende situaties *veranderingen* te bestuderen. 

Vele fenomenen zoals bewegingen, temperatuursverlopen, productievolumes, epidemies en hoogteprofielen van wielerwedstrijden kunnen beschreven worden als functies. *Afgeleiden* zeggen dan iets over de *veranderingen van de functiewaarden*. Hoe *verandert* de temperatuur, het geproduceerde volume, het aantal besmette personen als de tijd, de prijs of de sociale contacten veranderen?

In deze context begrijpen we functies dikwijls als een verband tussen een 'onafhankelijke' variabele (de tijd, de prijs, de lockdown maatregelen) en de 'afhankelijke' variabele (de temperatuur, de hoeveelheid geproduceerde producten of het aantal besmette personen). In wiskundige taal noemen we de onafhankelijke variabele typisch $x$ (maar soms ook $t$ als het over tijd gaat, of $s$, of andere letters) en de afhankelijke grootheid is dan een functie die we typisch $f$ noemen (maar soms ook $T$ als het over temperatuur gaat, of $v$ en $a$ als het snelheden en versnellingen zijn).

Afgeleiden geven informatie over de *verhouding* tussen *de verandering van de afhankelijke variabele* ten opzichte van *de verandering van de onafhankelijke variabele*. Als je de onafhankelijke variabele varieert, wat gebeurt er dan met de afhankelijke variabele? Stijgt die, of daalt die, en hoe snel of hoe traag gebeurt dat? 
Je kan hetzelfde ook begrijpen in termen van *gevoeligheid*: hoe gevoelig is de afhankelijke variabele voor een wijziging in de onafhankelijke variabele? 

## T3.2 -  Een inleidend voorbeeld

We illustreren eerst aan de hand van een voorbeeld wat we bedoelen met *veranderingen* van functiewaarden, en *verhoudingen* tussen *de verandering van de afhankelijke variabele* ten opzichte van *de verandering van de onafhankelijke variabele*.

In een punt $a+h$ in de buurt van $a$ is de functiewaarde van $f$ gelijk aan $f(a+h)$.
    We berekenen voor enkele steeds kleiner wordende $h$ telkens de functiewaarden in $a$ en in $a+h$ en verandering $f(a+h)-f(a)$ die hoort bij deze $h$.
    Zoals te verwachten was, wordt de verandering $f(a+h)-f(a)$ ook erg klein zodra $h$ klein wordt. Maar het is erg interessant op te merken dat de *verhouding* $\frac{f(a+h)-f(a)}{h}$ *niet* noodzakelijk klein wordt!
    
Laten we de functie $f(x) = x^2 + 1$ beschouwen, en bij $a=1$ en $a=3$ voor steeds kleiner wordende $h$ de verhouding $\frac{f(a+h)-f(a)}{h}$ berekenen. De resultaten vinden we in onderstaande tabellen, en de verschillende punten en verhoudingen worden in bijhorende interactieve figuur nog eens afgebeeld. 


| a |   h   |  a+h  | f(a) | f(a+h)  | f(a+h)-f(a) |f(a+h)-f(a)/h   |
 | --- | --- | --- | --- | ---| --- | --- |
		|	1 |   1   |   2   |  2  |   5     |      3      | 3       |
		|	1 |  0.5  |  1.5  |  2  |  3.25  |    1.25     | 2.5      |
		|	1 |  0.1  |  1.1  |  2  |  2.21  |    0.21     | 2.1      |
		|	1 | 0.01  | 1.01  |  2  | 2.0201 |  0.00201   | 2.01      |
		|	1 | 0.001 | 1.001 |  2  | 2.0020  |  0.00020   | 2.001     |
        
        
        
 | a |   h   |  a+h  | f(a) | f(a+h)  | f(a+h)-f(a) |f(a+h)-f(a)/h   |
 | --- | --- | --- | --- | ---| --- | --- |
			|3 |   1   |   4   |  10  |   17    |      7     |            7    |
			|3 |  0.5  |  3.5  |  10  |  13.25  |    3.25    |           6.5   |
			|3 |  0.1  |  3.1  |  10  |  10.61  |    0.61    |           6.1   |
			|3 | 0.01  | 3.01  |  10  | 10.0601 |   0.0601   |          6.01   |
			|3 | 0.001 | 3.001 |  10  | 10.006  |  0.006001  |          6.001  |

In [40]:
x = np.linspace(-1,2,100)



def animatie(a,h):


    def f(x):
        return x**2+1

    plt.plot(x, f(x), 'r',label='$f(x)=x^2+1$')

    plt.plot([a,a+h], [f(a),f(a)], 'k')
    plt.text(a+h/2, f(a)-0.2, 'h')

    plt.plot([a+h,a+h], [f(a),f(a+h)], 'k')
    plt.text(a+h+0.1,f(a)+(f(a+h)-f(a))/2, 'f(a+h)-f(a)')
    
    plt.plot([a,a+h], [f(a),f(a+h)], 'k')
    
    plt.plot([a], [f(a)], 'g',marker='o')
    plt.text(a-0.4, f(a)-0.05, '(a,f(a))',color='g')
    
    plt.plot([a+h], [f(a+h)], 'g',marker='o')
    plt.text(a+h-0.7, f(a+h)-0.05, '(a+h,f(a+h))',color='g')
    

    plt.title(r'$a={:.2f}$, $h={:.4f}$, $\dfrac{{f(a+h) - f(a)}}{{h}} ={:.4f} $'.format(a,h,(f(a+h)-f(a))/h))
    
    plt.ylim([0,5])
    plt.legend()

    
interact(animatie, h=FloatLogSlider(value=10,
    base=10,
    min=-3, # max exponent of base
    max=0, # min exponent of base
    step=0.25, # exponent step,
                                 description='h', style=style, layout=slider_layout),
        a=IntSlider(value=1,min=-1,max=1,step=1,description='a', style=style, layout=slider_layout));

interactive(children=(IntSlider(value=1, description='a', layout=Layout(width='50%'), max=1, min=-1, style=Sli…

 We stellen vast dat wanneer $h$ steeds kleiner wordt, of dus wanneer we de [limiet](./T2-Limieten.ipynb) $\lim\limits_{h\to0}$ nemen, de verhouding $\frac{f(a+h)-f(a)}{h}$ naar een bepaalde waarde convergeert. Deze waarde drukt uit hoe een kleine verandering van de onafhankelijke veranderlijke rond $a$ de waarde van $f$ beïnvloedt.

## T3.3 -  Definitie

$\require{tcolorbox}$
We noemen een functie $f$ *afleidbaar* of *differentieerbaar* in een punt $a$ als de limiet 

$$
	\lim_{h\to0} \dfrac{f(a+h)-f(a)}{h}
$$

bestaat.
    
>  ❗ In dat geval noemen we die limiet de **afgeleide van $f$ in $a$** en we noteren
	$$
	f'(a) \equiv \lim_{h\to0} \dfrac{f(a+h)-f(a)}{h}.
	$$
    


We berekenen nu als voorbeeld enkele afgeleiden van de eerder bestudeerde functie $f(x)=x^2+1$ rechtstreeks op basis van deze definitie. In de praktijk zal je deze afgeleiden echter altijd met rekenregels berekenen.

Bereken de afgeleide van $f(x) = x^2+1$ in het punt $a=3$:

$$
\begin{align}
f'(3) & = \underset {h \to  0} \lim \dfrac{f(3+h)-f(3)}{h} \\
      & = \underset {h \to  0} \lim \dfrac{\left((3+h)^2+1\right)-(3^2+1)}{h} \\
      & = \underset {h \to  0} \lim \dfrac{9+6h+h^2 + 1 - 9 -1}{h} \\
      & = \underset {h \to  0} \lim \dfrac{6h+h^2}{h} = \underset {h \to  0} \lim (6+h) = 6
\end{align}
$$

Bereken de afgeleide van $f(x) = x^2+1$ in een willekeurig punt $a$:

$$
\begin{align}
f'(a) & = \underset {h \to  0} \lim \dfrac{f(a+h)-f(a)}{h} \\
      & = \underset {h \to  0} \lim \dfrac{(a+h)^2+1-(a^2+1)}{h} \\
      & = \underset {h \to  0} \lim \dfrac{a^2+2ah+h^2 + 1 - a^2 -1}{h} \\
      & = \underset {h \to  0} \lim \dfrac{2ah+h^2}{h} = \underset {h \to  0} \lim (2a+h) = 2a
\end{align}
$$

In dit laatste voorbeeld hebben we de afgeleide van f in *elk* punt $a$ berekend. Op die manier verkrijgen we een nieuwe functie die we de afgeleide functie noemen: 

Voor een functie $f(x)$ definiëren we de **afgeleide functie van $f(x)$**, genoteerd als $f'(x)$ (ook als $\dfrac{df}{dx}$ of $\dfrac{d}{dx}f(x)$ of $D f(x)$ of $\dot{f}$). Deze afgeleide functie, $f'(x)$, geeft in $x$ de waarde van de afgeleide van $f(x)$ in $x$. 

We hebben boven gezien dat voor willekeurige $a$, de afgeleide van $f(x)=x^2+1$ in $a$ gelijk is aan $2a$. De afgeleide functie is dus $f'(x)=2x$.

AFBEELDING VAN FUNCTIE EN AFGELEIDE FUNCTIE

## T3.4 -  De raaklijn

Bij het inleidend voorbeeld hebben we onderzocht hoe de coördinaten $(a,f(a))$ en $(a+h,f(a+h))$ zich tot elkaar verhouden. Laten we de vergelijking van de rechte onderzoeken die precies door deze coördinaten loopt. Dankzij [T1-Functies](./T1-Functies.ipynb) weten we dat deze rechte gegeven wordt door:

$$
y -f(a) = \dfrac{f(a+h)-f(a)}{a+h-a} \left( x - a \right).
$$

Stel nu dat we $(a+h,f(a+h))$ opnieuw steeds dichter bij $(a,f(a))$ laten naderen, en dus de limiet $\lim\limits_{h\to0}$ nemen, dan krijgen we:

$$
\begin{align}
&& \lim_{h\to0} \left( y -f(a) \right)&= \lim_{h\to0} \left(\dfrac{f(a+h)-f(a)}{h} \left( x - a \right) \right),\\
&\Leftrightarrow&  y -f(a)&= \lim_{h\to0} \left(\dfrac{f(a+h)-f(a)}{h} \right)\cdot \lim_{h\to0} \left( x - a  \right),\\
&\Leftrightarrow&  y -f(a)&= \lim_{h\to0} \left(\dfrac{f(a+h)-f(a)}{h} \right)\left( x - a \right),\\
&\Leftrightarrow&  y -f(a)&= f'(a) \left( x - a \right),
\end{align}
$$

waar we eerst gebruik maakten van de product-rekenregel voor [limieten](./T2-Limieten.ipynb), en vervolgens de definitie van een afgeleide herkenden.

In [44]:
x = np.linspace(-1,2,100)



def animatie(a):


    def f(x):
        return x**2+1
    
    def rl(x):
        return (2*a)*(x-a)+f(a)

    plt.plot(x, f(x), 'r',label='$f(x)=x^2+1$')
    plt.plot(x, rl(x), 'k',label=r'$y-{0} = {1}\cdot(x-{2})$'.format(f(a),2*a,a))
    
    plt.plot([a], [f(a)], 'g',marker='o')
    
    
    
    plt.ylim([0,5])
    plt.legend()

    
interact(animatie, 
        a=FloatSlider(value=1,min=-1,max=1,step=0.5, description='a', style=style, layout=slider_layout));

interactive(children=(FloatSlider(value=1.0, description='a', layout=Layout(width='50%'), max=1.0, min=-1.0, s…

De *afgeleide* in een punt is dus de *richtingscoëfficiënt* van de raaklijn aan de grafiek in dat punt (Zie voorbeeld \ref{vb:helling_raaklijn}). De vergelijking van de raaklijn aan de grafiek van $f$ in het punt $(a,f(a))$ is dus
		$$
		y-f(a) = f'(a)(x-a)
		$$

## T3.5 -  Rekenregels

We kunnen steeds op bovenstaande manier voor eender welke functie de afgeleide functie berekenen, als ze bestaat, maar deze methode is erg omslachtig. Gelukkig bestaan er eenvoudige rekenregels die we op een functie kunnen toepassen om de afgeleide functie te verkrijgen. Het is niet de bedoeling dat je deze rekenregels vanbuiten kent. Indien er elders in deze cursus gebruik wordt gemaakt van een van deze regels, kun je ze hier even opsnorren.

$$
\begin{align}
&&\dfrac{d}{dx} c  &=  0,&&\text{ met $c$ een constante} \\
&&\dfrac{d}{dx}(x^r)   &=   rx^{r-1},&&\text{ met $r$ een constante} \\
\\
&&\dfrac{d}{dx}(c f)   &=   c\dfrac{d}{dx}(f),&&\text{ met $c$ een constante} \\
&&\dfrac{d}{dx}(f+g)   &=   \dfrac{d}{dx}(f)+\dfrac{d}{dx}(g)\\
\\
&\text{afgeleiden van bijzondere functies:}\\
&&\dfrac{d}{dx}(e^x)   &=   e^x\\
&&\dfrac{d}{dx}(a^x)   &=   a^x \ln(a),&&\text{ met $a$ een constante} \\
&&\dfrac{d}{dx}(\ln(x))   &=   \dfrac{1}{x}\\
&&\dfrac{d}{dx}(\log_a(x))   &=   \dfrac{1}{x \ln(a)},&&\text{ met $a$ een constante} \\
\\
&&\dfrac{d}{dx}(\sin x)   &=   \cos x\\
&&\dfrac{d}{dx}(\cos x)   &=   -\sin x\\
\\
&\text{quotiënt- en productregels:}\\
&&\dfrac{d}{dx}(f\cdot g)   &=   \dfrac{d}{dx}(f)\cdot g+f\cdot \dfrac{d}{dx}(g)\\
&&\dfrac{d}{dx}(\dfrac{f}{g})   &=   \dfrac{\dfrac{d}{dx}(f)\cdot g-f\cdot \dfrac{d}{dx}(g)}{g^2} \\
\\
&\text{de kettingregel:}\\
&&\dfrac{d}{dx}(f(g(x)))&=f'(g(x))\cdot \dfrac{d}{dx}g(x)
\end{align}
$$

## T3.6 -   Enkele voorbeelden

#### Positie, snelheid en versnelling

#### Diffusie flux?

 ## Referentie
 
 De tekst van dit hoofdstuk is deels gebaseerd op https://set.kuleuven.be/voorkennis/zomercursus/zomercursusZ